In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('https://query.data.world/s/rvag7de6nxmo5pradcjaxlx67sinj6')

In [3]:
df_melted = df.melt(df.columns[:2], df.columns[2:], var_name = 'Year', value_name = 'Spend')

df_melted['Type'] = df_melted['Type'].map(lambda x: 'Internet' if x == 'Pure-Play Internet' else x)

df_melted = df_melted.loc[(df_melted['Type'] != 'Total TV') & (df_melted['Type'] != 'Media total')]

df_reduced = df_melted.groupby(['Type', 'Year'])['Spend'].sum().reset_index()

In [4]:
spend_weights = [0] + [1 / (1 + np.exp(-x)) for x in np.arange(-5, 5.5, 0.5)] + [1]
year_incriments = [i/len(spend_weights) for i in range(len(spend_weights))]

df_reduced['Past Year'] = df_reduced['Year'].map(lambda x: x - 1)
df_merged = pd.merge(df_reduced, df_reduced, left_on = ['Type', 'Year'], right_on = ['Type', 'Past Year'])

df_merged.rename(columns = {
    'Type': 'type',
    'Year_x': 'year',
    'Spend_x': 'spend',
    'Spend_y': 'next_spend'}, inplace = True)

In [5]:
spend_weights = [1 / (1 + np.exp(-x)) for x in np.arange(-5, 5.5, 0.5)] + [1]

adjusted_spends = []
for index, row in df_merged.iterrows():
    for i in range(1, len(spend_weights) + 1):
        adjusted_year = row['year'] + i / len(spend_weights)
        adjusted_spend = row['spend'] + ((row['next_spend'] - row['spend']) * spend_weights[i - 1])
        adjusted_spends.append({'type': row['type'], 'year': adjusted_year, 'spend': adjusted_spend})

df_adjusted = pd.DataFrame(adjusted_spends)

In [21]:
max_year_spend = df_reduced.groupby(['Year'])['Spend'].sum().max()
total_year_spend_gaps = (max_year_spend - df_adjusted.groupby(['year'])['spend'].sum()).reset_index()
dummy_year_spends = []
for index, row in total_year_spend_gaps.iterrows():
    for dummy_type in ['dummy_1', 'dummy_2']:
        dummy_spend = row['spend'] / 2
        dummy_year_spends.append({'type': dummy_type, 'year': row['year'], 'spend': dummy_spend})

dummy_year_spends = pd.DataFrame(dummy_year_spends)

df_final = pd.concat([df_adjusted, dummy_year_spends])

df_final = df_final.reset_index().drop(columns = ['index'])

In [25]:
df_final

,type,year,spend
0,Internet,2012.045455,5.683022e+10
1,Internet,2012.090909,5.686885e+10
2,Internet,2012.136364,5.693182e+10
3,Internet,2012.181818,5.703372e+10
4,Internet,2012.227273,5.719669e+10
...,...,...,...
2107,dummy_2,2023.909091,9.042968e+07
2108,dummy_1,2023.954545,5.508651e+07
2109,dummy_2,2023.954545,5.508651e+07
2110,dummy_1,2024.000000,0.000000e+00


In [27]:
df_final.to_csv('W46.csv', index = False)